# Notebook to plot the helicity angles of $a_{0}$

In [11]:
import ROOT
import numpy as np
import my_library.common_analysis_tools as ct
import my_library.constants as constants
import my_library.kinematic_cuts as kcuts
import pandas as pd

In [ ]:
def get_gamma_cm(ey: float, s: float) -> float:
    # type: (float, float) -> float
    return (ey + 0.938272)/np.sqrt(s)
# def get_gamma_cm(ey, s):
#     return (ey + 0.938272)/np.sqrt(s)

def get_beta_cm(ey: float) -> float:
    return ey/(ey + 0.938272)


def get_beta(p: float, e: float) -> float:
    return p/e


def get_gamma(e: float, m: float) -> float:
    return e/m


In [ ]:
def get_cm_vector(e: float, px: float, py: float, pz: float, ey: float, s: float):
    gamma = get_gamma_cm(ey, s)
    beta = get_beta_cm(ey)
    vec_cm_rotaton = np.array([[gamma, 0, 0, -1*beta*gamma],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [-1*beta*gamma, 0, 0, gamma]])
    vec_lab = np.array([e, px, py, pz]).reshape(4,1)
    return (vec_cm_rotaton @ vec_lab).reshape(1,4)


In [ ]:
def get_rest_frame_vec(p4_decay_cm: float, p4_rest_cm: float, m_rest: float):
    """e in 0th element, px in 1st, py in 2nd, pz in 3rd"""
    beta = get_beta(np.sqrt(p4_rest_cm[1]*p4_rest_cm[1] + p4_rest_cm[2]*p4_rest_cm[2] + p4_rest_cm[3]*p4_rest_cm[3]), p4_rest_cm[0], p4_rest_cm[0])
    gamma = get_gamma(p4_rest_cm[0], m_rest)
    beta_x = get_beta(p4_rest_cm[1], p4_rest_cm[0])
    beta_y = get_beta(p4_rest_cm[2], p4_rest_cm[0])
    beta_z = get_beta(p4_rest_cm[3], p4_rest_cm[0])

    p4_decay_cm = p4_decay_cm.reshape(-1, 1)

    rest_rotation = np.array([[gamma,         -gamma*beta_x,                             -gamma*beta_y,                             -gamma*beta_z                            ],
                              [-gamma*beta_x, 1 + (gamma - 1)*beta_x*beta_x/(beta*beta), (gamma - 1)*beta_x*beta_y/(beta*beta),     (gamma - 1)*beta_x*beta_z/(beta*beta)    ],
                              [-gamma*beta_y, (gamma - 1)*beta_y*beta_x/(beta*beta),     1 + (gamma - 1)*beta_y*beta_y/(beta*beta), (gamma - 1)*beta_y*beta_z/(beta*beta)    ],
                              [-gamma*beta_z, (gamma - 1)*beta_z*beta_x/(beta*beta),     (gamma - 1)*beta_z*beta_y/(beta*beta),     1 + (gamma - 1)*beta_z*beta_z/(beta*beta)]])

    return (rest_rotation @ p4_decay_cm).reshape(1, -1)

In [10]:
rdf_pipkmks = ct.get_dataframe('pipkmks', 'spring', 'data')
np_pipkmks = rdf_pipkmks.AsNumpy()

In [12]:
df_pipkmks = pd.DataFrame(np_pipkmks)